<a href="https://colab.research.google.com/github/Jeong-ki-hun/Section2_Project/blob/main/%08AI_11_%EC%A0%95%EA%B8%B0%ED%9B%88(Section2_Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install category_encoders
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 6.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 32.6 MB/s 


# 라이브러리 호출

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rc('font', family='NanumBarunGothic') 
sns.set_style("whitegrid")

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import plot_confusion_matrix
import sklearn
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder, OneHotEncoder
from scipy.stats import randint, uniform
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import shap
from sklearn.linear_model import ElasticNetCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
df = pd.read_csv('/content/drive/MyDrive/국토교통부_표준주택가격_20220101.csv',encoding='cp949')

데이터는 공공데이터포털에 국토교통부_표준주택가격 데이터를 사용했습니다.

# EDA

컬럼설명

In [21]:
df.columns

Index(['시도', '시군구', '일련번호', '소재지', '도로명주소', '지목', '건물용도', '대지면적', '연면적',
       '건축면적', '용적률', '건폐율', '동수', '층수(지상_지하)', '용도지역', '용도지구', '도로교통', '형상지세',
       '주건물구조', '사용승인일', '주위환경', '주택공시가격'],
      dtype='object')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 22 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   시도         239809 non-null  object 
 1   시군구        238774 non-null  object 
 2   일련번호       240000 non-null  int64  
 3   소재지        240000 non-null  object 
 4   도로명주소      240000 non-null  object 
 5   지목         240000 non-null  object 
 6   건물용도       240000 non-null  object 
 7   대지면적       240000 non-null  float64
 8   연면적        240000 non-null  float64
 9   건축면적       240000 non-null  float64
 10  용적률        240000 non-null  float64
 11  건폐율        240000 non-null  float64
 12  동수         240000 non-null  int64  
 13  층수(지상_지하)  240000 non-null  object 
 14  용도지역       240000 non-null  object 
 15  용도지구       56279 non-null   object 
 16  도로교통       240000 non-null  object 
 17  형상지세       240000 non-null  object 
 18  주건물구조      240000 non-null  object 
 19  사용승인일      240000 non-n

In [87]:
df.isnull().sum()

시도              191
시군구            1226
일련번호              0
소재지               0
도로명주소             0
지목                0
건물용도              0
대지면적              0
연면적               0
건축면적              0
용적률               0
건폐율               0
동수                0
층수(지상_지하)         0
용도지역              0
용도지구         183721
도로교통              0
형상지세              0
주건물구조             0
사용승인일             0
주위환경              0
주택공시가격            0
지상                0
지하                0
dtype: int64

In [90]:
df.drop(columns=['용도지구'],axis=1,inplace=True)


In [94]:
df.drop(columns=['일련번호'],axis=1,inplace=True)


In [74]:
df['지상'] = df['층수(지상_지하)'].str.split('_').str[0]
df['지하'] = df['층수(지상_지하)'].str.split('_').str[1]

In [83]:
df['지상']=df['지상'].str.replace('(','')
df['지하'] = df['지하'].str.replace(')','')

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [96]:
df.corr()

,대지면적,연면적,건축면적,용적률,건폐율,동수,사용승인일,주택공시가격
대지면적,1.000000,0.013620,0.369777,-0.403268,-0.603125,0.160769,0.129400,-0.027866
연면적,0.013620,1.000000,0.656731,0.710718,0.389438,-0.174757,0.318829,0.442878
건축면적,0.369777,0.656731,1.000000,0.198028,0.095847,0.153017,0.364345,0.231549
용적률,-0.403268,0.710718,0.198028,1.000000,0.793509,-0.277314,0.209449,0.315695
건폐율,-0.603125,0.389438,0.095847,0.793509,1.000000,-0.174419,0.093852,0.195081
동수,0.160769,-0.174757,0.153017,-0.277314,-0.174419,1.000000,-0.291982,-0.188200
사용승인일,0.129400,0.318829,0.364345,0.209449,0.093852,-0.291982,1.000000,0.153696
주택공시가격,-0.027866,0.442878,0.231549,0.315695,0.195081,-0.188200,0.153696,1.000000


In [88]:
df['용도지구']

0         자연경관지구
1         자연경관지구
2         자연경관지구
3         자연경관지구
4         자연경관지구
           ...  
239995    자연취락지구
239996    자연취락지구
239997    자연취락지구
239998    자연취락지구
239999       NaN
Name: 용도지구, Length: 240000, dtype: object

In [24]:
df.iloc[:1]

,시도,시군구,일련번호,소재지,도로명주소,지목,건물용도,대지면적,연면적,건축면적,...,동수,층수(지상_지하),용도지역,용도지구,도로교통,형상지세,주건물구조,사용승인일,주위환경,주택공시가격
0,서울특별시,종로구,1,청운동 4-18,자하문로36길 34,대,단독주택,1193.4,499.24,268.7,...,1,(2_1),제1종일반주거지역,자연경관지구,세로(가),정방형,연와조,1994,고급 주택지대,4952000000


In [28]:
target = df['주택공시가격']
features = df.drop(columns=['주택공시가격'])

In [29]:
features

,시도,시군구,일련번호,소재지,도로명주소,지목,건물용도,대지면적,연면적,건축면적,...,건폐율,동수,층수(지상_지하),용도지역,용도지구,도로교통,형상지세,주건물구조,사용승인일,주위환경
0,서울특별시,종로구,1,청운동 4-18,자하문로36길 34,대,단독주택,1193.4,499.24,268.70,...,22.52,1,(2_1),제1종일반주거지역,자연경관지구,세로(가),정방형,연와조,1994,고급 주택지대
1,서울특별시,종로구,2,청운동 8-16,자하문로35길 12,대,단독주택,1046.3,466.35,181.06,...,17.30,1,(2_0),제1종일반주거지역,자연경관지구,세로(가),부정형,철근콘크리트조(RC조 RS조),1965,고급 주택지대
2,서울특별시,종로구,3,청운동 12-9,자하문로36길 11,대,단독주택,237.0,159.74,91.54,...,38.62,1,(2_0),제1종일반주거지역,자연경관지구,소로한면,부정형,연와조,1967,고급 주택지대
3,서울특별시,종로구,4,청운동 15-10,창의문로1길 20-5,대,단독주택,637.5,508.97,189.33,...,29.70,2,(2_1),제1종일반주거지역,자연경관지구,세로(가),사다리형,철근콘크리트조(RC조 RS조),1978,고급 주택지대
4,서울특별시,종로구,5,청운동 15-26,창의문로1길 7,대,단독주택,62.8,33.06,33.06,...,52.64,1,(1_0),자연녹지지역,자연경관지구,세로(불),사다리형,시멘트블럭조,1963,기존 주택지대
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,제주특별자치도,서귀포시,1874,표선면 토산리 936-3,표선면 토산망동로 273-11,대,단독주택,410.0,78.45,78.45,...,19.13,2,(1_0),계획관리지역,자연취락지구,중로각지,부정형,시멘트블럭조,1985,지방도변 농촌지대
239996,제주특별자치도,서귀포시,1875,표선면 토산리 1363-4,표선면 중산간동로 5560,대,단독주택,413.0,88.00,88.00,...,21.31,2,(1_0),계획관리지역,자연취락지구,중로한면,사다리형,시멘트벽돌조,1993,지방도변 농촌지대
239997,제주특별자치도,서귀포시,1876,표선면 토산리 1416-3,표선면 토산중앙로 292,대,단독주택,217.0,78.76,78.76,...,36.29,1,(1_0),계획관리지역,자연취락지구,소로한면,세로장방형,시멘트블럭조,1985,국도주변 농촌지대
239998,제주특별자치도,서귀포시,1877,표선면 토산리 1826-1,표선면 토산중앙로287번길 29-6,대,단독주택,602.0,98.30,98.30,...,16.33,1,(1_0),계획관리지역,자연취락지구,세로(가),사다리형,시멘트벽돌조,1998,지방도변 농촌지대


In [32]:
pd.DataFrame(features_encoded)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,8.0,177.0,0.0,101420.0,180625.0,4.0,3.0,6052.0,36800.0,20042.0,...,1887.0,0.0,6.0,16.0,11.0,5.0,5.0,95.0,118.0,2.0
1,8.0,177.0,1.0,101428.0,180623.0,4.0,3.0,5886.0,34980.0,15610.0,...,1365.0,0.0,5.0,16.0,11.0,5.0,1.0,139.0,89.0,2.0
2,8.0,177.0,2.0,101417.0,180624.0,4.0,3.0,2040.0,13331.0,6889.0,...,3497.0,0.0,5.0,16.0,11.0,8.0,1.0,95.0,91.0,2.0
3,8.0,177.0,3.0,101418.0,202522.0,4.0,3.0,5077.0,37231.0,16277.0,...,2605.0,1.0,6.0,16.0,11.0,5.0,2.0,139.0,102.0,2.0
4,8.0,177.0,4.0,101419.0,202523.0,4.0,3.0,320.0,879.0,1092.0,...,4899.0,0.0,2.0,13.0,11.0,6.0,2.0,80.0,87.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,14.0,91.0,1873.0,227403.0,218597.0,4.0,3.0,3724.0,5216.0,5580.0,...,1548.0,1.0,2.0,1.0,12.0,9.0,1.0,80.0,109.0,83.0
239996,14.0,91.0,1874.0,227391.0,218590.0,4.0,3.0,3750.0,6171.0,6535.0,...,1766.0,1.0,2.0,1.0,12.0,10.0,2.0,65.0,117.0,83.0
239997,14.0,91.0,1875.0,227392.0,218598.0,4.0,3.0,1840.0,5247.0,5611.0,...,3264.0,0.0,2.0,1.0,12.0,8.0,3.0,80.0,109.0,13.0
239998,14.0,91.0,1876.0,227393.0,218601.0,4.0,3.0,4937.0,7201.0,7565.0,...,1268.0,0.0,2.0,1.0,12.0,5.0,2.0,65.0,122.0,83.0


In [61]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [62]:
from sklearn.compose import TransformedTargetRegressor

In [63]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder=OrdinalEncoder()
X_train =ordinal_encoder.fit_transform(X_train)
X_test =ordinal_encoder.fit_transform(X_test)

In [56]:
X_train

OrdinalEncoder()

In [58]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [65]:
pipe_lin = make_pipeline(
    StandardScaler(),
    LinearRegression(n_jobs=-1)
)

pipe_lin.fit(X_train, y_train)
y_pred = np.exp(pipe_lin.predict(X_test))
evaluate('Linear Regression', y_test, y_pred)

ValueError: ignored